DraftKings NFL Constraint Satisfaction
===

See also [this blog post](https://levelup.gitconnected.com/dfs-lineup-optimizer-with-python-296e822a5309).

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
import os
import sys
from datetime import datetime

In [6]:
import pulp

### Load in the weekly data

In [87]:
df = pd.read_csv('DKSalaries.csv')
len(df)

476

In [88]:
df.sample(n=5)

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
235,RB,Jalen Richard (15667795),Jalen Richard,15667795,RB/FLEX,4000,TB@LV 10/25/2020 04:05PM ET,LV,5.38
439,TE,Sean McKeon (15642872),Sean McKeon,15642872,TE/FLEX,2500,DAL@WAS 10/25/2020 01:00PM ET,DAL,0.00
267,TE,Greg Olsen (15642778),Greg Olsen,15642778,TE/FLEX,3600,SEA@ARI 10/25/2020 08:20PM ET,SEA,7.00
249,RB,Jaylen Samuels (15642324),Jaylen Samuels,15642324,RB/FLEX,4000,PIT@TEN 10/25/2020 01:00PM ET,PIT,0.93
313,WR,Chris Hogan (15642702),Chris Hogan,15642702,WR/FLEX,3000,BUF@NYJ 10/25/2020 01:00PM ET,NYJ,5.16


In [89]:
df[df.Name.map(lambda name: 'Haskins' in name)]

,Position,Name + ID,Name,ID,Roster Position,Salary,Game Info,TeamAbbrev,AvgPointsPerGame
89,QB,Dwayne Haskins Jr. (15642203),Dwayne Haskins Jr.,15642203,QB,5000,DAL@WAS 10/25/2020 01:00PM ET,WAS,15.14


In [90]:
# trim any postponed games, since those can't be included in a lineup
df = df[df['Game Info'] != 'Postponed']
len(df)

476

In [91]:
exclude_list = ['Dwayne Haskins Jr.', 'Jamison Crowder', 'Allen Lazard', 'Aaron Jones', 'Dak Prescott']
df = df[~df['Name'].isin(exclude_list)]
len(df)

471

In [92]:
# this is equivalent to an extra constraint that requires playing only players with a minimum cost
# does not apply to DST, since that's kind of a special category
df = df[(df.Salary >= 4000)|(df['Roster Position'] == 'DST')]
len(df)

272

### Create the constraint problem

Goal: maximize AvgPointsPerGame

 - TotalPlayers = 10
 - TotalSalary <= 50000
 - TotalPosition_WR = 3
 - TotalPosition_RB = 3
 - TotalPosition_TE = 1
 - TotalPosition_QB = 1
 - TotalPosition_FLEX = 1
 - TotalPosition_DST = 1
 - Each player in only one position (relevant only for FLEX)
 

In [93]:
prob = pulp.LpProblem('DK_NFL_weekly', pulp.LpMaximize)

In [94]:
player_vars = [pulp.LpVariable(f'player_{row.ID}', cat='Binary') for row in df.itertuples()]

In [95]:
# total assigned players constraint
prob += pulp.lpSum(player_var for player_var in player_vars) == 9

In [96]:
# position constraints
# TODO fix this, currently won't work
# as it makes the problem infeasible
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'QB') == 1
prob += get_position_sum(player_vars, df, 'DST') == 1

# to account for the FLEX position, we allow additional selections of the 3 FLEX-eligible roles
prob += get_position_sum(player_vars, df, 'RB') >= 2
prob += get_position_sum(player_vars, df, 'WR') >= 3
prob += get_position_sum(player_vars, df, 'TE') >= 1

In [97]:
# total salary constraint
prob += pulp.lpSum(df.Salary.iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [98]:
# finally, specify the goal
prob += pulp.lpSum([df.AvgPointsPerGame.iloc[i] * player_vars[i] for i in range(len(df))])

In [99]:
prob.solve()

1

In [100]:
pulp.LpStatus[prob.status]

'Optimal'

In [101]:
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Name, row.TeamAbbrev, row.Salary, row.AvgPointsPerGame)
        total_salary_used += row.Salary
        mean_AvgPointsPerGame += row.AvgPointsPerGame
mean_AvgPointsPerGame /= 10  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

QB Russell Wilson SEA 8000 31.88
RB/FLEX Alvin Kamara NO 7900 30.12
TE/FLEX George Kittle SF 6500 21.92
WR/FLEX Chase Claypool PIT 5700 18.32
RB/FLEX Melvin Gordon III DEN 5500 17.4
WR/FLEX Corey Davis TEN 4800 14.87
TE/FLEX Jonnu Smith TEN 4700 14.48
WR/FLEX Tim Patrick DEN 4600 13.8
DST Cardinals  ARI 2300 7.0


(50000, 16.979000000000003)